In [9]:
import spacy
import json
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from spacy.training.example import Example
from seqeval.metrics import classification_report

In [10]:
# Load model yang sudah dilatih
model_path = "models2/model_iter60"
nlp = spacy.load(model_path)
print("Model berhasil dimuat.")

Model berhasil dimuat.


In [11]:
# 2. Load data test
with open("data_test.json", "r", encoding="utf-8") as f:
    data_json = json.load(f)
    TEST_DATA = data_json["annotations"]

In [12]:
# 3. Evaluasi model
true_entities_list = []
pred_entities_list = []

for text, annot in TEST_DATA:
    doc = nlp(text)
    
    # Konversi prediksi model ke format list tag per token
    pred_entities = []
    for token in doc:
        if token.ent_type_:
            # Gunakan skema BIO yang benar
            if token.ent_iob_ == "B":
                pred_entities.append(f"B-{token.ent_type_}")
            else:  # token.ent_iob_ == "I"
                pred_entities.append(f"I-{token.ent_type_}")
        else:
            pred_entities.append("O")
    
    # Konversi ground truth ke format list tag per token
    true_entities = ["O"] * len(doc)
    for start, end, label in annot["entities"]:
        # Cari token yang sesuai dengan span entitas
        for token_idx, token in enumerate(doc):
            # Jika token ada dalam span entitas
            if start <= token.idx < end:
                # Token pertama dalam entitas
                if token.idx == start or (token_idx > 0 and doc[token_idx-1].idx < start):
                    true_entities[token_idx] = f"B-{label}"
                else:
                    true_entities[token_idx] = f"I-{label}"
    
    true_entities_list.append(true_entities)
    pred_entities_list.append(pred_entities)

# Evaluasi menggunakan seqeval
print("Hasil Evaluasi Model:")
print(classification_report(true_entities_list, pred_entities_list))

Hasil Evaluasi Model:
              precision    recall  f1-score   support

         APP       0.88      0.88      0.88       357
        DESC       0.83      0.77      0.80       142

   micro avg       0.87      0.85      0.86       499
   macro avg       0.86      0.82      0.84       499
weighted avg       0.87      0.85      0.86       499

